<a href="https://colab.research.google.com/github/RogerHeederer/DL_Entry_Pytorch/blob/master/Bi_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ref : pytorch로 시작하는 딥 러닝 입문

In [1]:
import numpy as np

timesteps = 10
input_size = 4
hidden_size = 8

inputs = np.random.random((timesteps, input_size))

hidden_state_t = np.zeros((hidden_size, ))

In [2]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
Wx = np.random.random((hidden_size, input_size)) # (8,4) 2d 텐서 생성. 입력에 대한 가중치
Wh = np.random.random((hidden_size, hidden_size)) #(8,8) 2d 텐서 생성. 은닉 상태에 대한 가중치
b = np.random.random((hidden_size,)) #(8,) 1d 텐서 생성. bias

In [4]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [6]:
total_hidden_states = []

for input_t in inputs: #각 시점에 따라서 입력값이 입력된다
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) #Wx*Xt + Wh*Ht-1 + b
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태 값을 축적
  print(np.shape(total_hidden_states)) #각 시점 t별 메모리 셀의 출력 형태 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0)
print("------축척 히든 스테이트-----")
print(total_hidden_states)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
------축척 히든 스테이트-----
[[0.99975712 0.99988215 0.99999522 0.9999987  0.99998433 0.99999562
  0.99928123 0.99996789]
 [0.99957377 0.99972855 0.99999545 0.99999721 0.99996615 0.99999315
  0.99878689 0.99995927]
 [0.9996316  0.99984825 0.99999672 0.99999694 0.99998386 0.99999518
  0.99940961 0.99995581]
 [0.9995663  0.99977788 0.99999466 0.99999716 0.99996933 0.99999262
  0.99872889 0.99996024]
 [0.99955127 0.99962359 0.99999659 0.99999649 0.99995717 0.99999479
  0.99907051 0.99995187]
 [0.99911762 0.99958689 0.99999298 0.9999915  0.99992573 0.99998872
  0.99790458 0.99994398]
 [0.99953447 0.99985395 0.99999302 0.99999643 0.99997407 0.99999276
  0.99887015 0.99995795]
 [0.99957657 0.99965201 0.99999729 0.99999681 0.999969   0.99999444
  0.9991426  0.99995407]
 [0.99949653 0.99964457 0.99999682 0.99999615 0.99996651 0.99999261
  0.99887616 0.99995375]
 [0.99972344 0.99972697 0.99999691 0.99999834 0.99997102 0.99999637
  

#파이토치로 구현하기 nn.RNN()

In [7]:
import torch
import torch.nn as nn

In [8]:
input_size = 5
hidden_szie = 8

In [9]:
#(batch size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [10]:
cell = nn.RNN(input_size, hidden_size, batch_first=True) #입력 텐서의 첫번째 차원이 배치 크기임을 알려줌

In [11]:
outputs, _status = cell(inputs)
print(outputs.shape)

torch.Size([1, 10, 8])


10번의 시점 동안 8차원의 은닉 상태가 출력되었음을 확인함

In [12]:
print(_status.shape) #맘지막 시점의 은닉 상태 크기 확인

torch.Size([1, 1, 8])


#깊은 순환 신경망

그림처럼 layer를 2층으로 쌓아보겠다.

<img src = "https://drive.google.com/uc?export=view&id=1wMI5iT6EFsmJa8jpFepGMe4gj2tPBwL6" align=left>

In [13]:
#(batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [19]:
cell_2 = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
outputs, _status = cell_2(inputs)

In [20]:
print(outputs.shape) # 위에 1층으로 구성한 RNN 셀과 다르지 않다

torch.Size([1, 10, 8])


In [21]:
print(_status.shape) # 1층 구성 RNN과 다름. (층의 개수, 배치 크기, 은닉 상태의 크기)

torch.Size([2, 1, 8])


#양방향 순환 신경만 구현 Bidirectional RNN

<img src = "https://drive.google.com/uc?export=view&id=1mFzNJt2e0jPF7VIW05SzMh0kd4RmygMg" align=left>

In [22]:
inputs = torch.Tensor(1, 10, 5)

In [23]:
cell_bi_2 = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True, bidirectional=True)

In [26]:
outputs, _status = cell_bi_2(inputs)

In [27]:
print(outputs.shape) # 배치 크기, 시퀀스 길이, 은닉 상태 크기 * 2(은닉상태값이 cancat됨)

torch.Size([1, 10, 16])


In [28]:
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([4, 1, 8])
